In [1]:
import pandas as pd 
from scipy import stats
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter, MaxNLocator

Step 1) load the datafiles into dataframes to better handling and joining for the KS tests.

In [2]:
#Colony dataframes
bishayee_colony = pd.read_excel("Bishayee Colony Counts 10.27.97-3.8.01.xlsx", skiprows=2,na_values=" ")
outside3_colony = pd.read_excel("Outside Lab 3.Colony Counts.2.4.10-5.21.12.xlsx", skiprows=1)
others_colony = pd.read_excel("Other Investigators in Lab.Colony Counts.4.23.92-11.27.02.xlsx", skiprows=1)
#Coulter dataframes
bishayee_coulter = pd.read_excel("Bishayee Coulter Counts.10.20.97-7.16.01.xlsx", skiprows=1)
others_coulter = pd.read_excel("Other Investigators in Lab.Coulter Counts.4.15.92-5.21.05.xlsx", skiprows=1)
    #This data file had no header, introduced manually
outside1_coulter = pd.read_excel("Outside Lab 1.Coulter Counts.6.7.91-4.9.99.xlsx")
outside1_coulter.columns = ["Experiment", "C1", "C2", "C3","Average","Date"]
outside2_coulter = pd.read_excel("Outside Lab 2.Coulter Counts.6.6.08-7.7.08.xlsx", skiprows=1)

The following step is to verify the magnitudes of the data in the sets for the coulter and the colony. From the description of the experimental set-up, it seems that the count of the cells using the coulter and the colony count are actually individual experiments with different methods. Hence, they should be analyzed independently

In [3]:
bishayee_colony.size

9534

In [4]:
bishayee_colony["Inv"] = "Z"

In [5]:
bishayee_colony.head(10)

ISOTOPE  Bate # B0/B00       Date  col1  col2   col3     average Inv
0     131I           8405 1997-10-27    78    91   93.0   87.333333   Z
1  no DMSO           8405 1997-10-27    90    88   90.0   89.333333   Z
2      NaN           8405 1997-10-27    80    66   69.0   71.666667   Z
3      NaN           8405 1997-10-27    63    67   71.0   67.000000   Z
4      NaN           8405 1997-10-27    44    58   64.0   55.333333   Z
5      NaN           8405 1997-10-27    38    53   51.0   47.333333   Z
6      NaN           8405 1997-10-27   247   264  258.0  256.333333   Z
7      NaN           8405 1997-10-27    46    24   27.0   32.333333   Z
8      NaN           8405 1997-10-27    64    63   61.0   62.666667   Z
9      NaN           8405 1997-10-27    77    82   98.0   85.666667   Z

In [6]:
bishayee_colony.drop('ISOTOPE', axis=1, inplace=True)

In [7]:
bishayee_colony.head(10)

Bate # B0/B00       Date  col1  col2   col3     average Inv
0           8405 1997-10-27    78    91   93.0   87.333333   Z
1           8405 1997-10-27    90    88   90.0   89.333333   Z
2           8405 1997-10-27    80    66   69.0   71.666667   Z
3           8405 1997-10-27    63    67   71.0   67.000000   Z
4           8405 1997-10-27    44    58   64.0   55.333333   Z
5           8405 1997-10-27    38    53   51.0   47.333333   Z
6           8405 1997-10-27   247   264  258.0  256.333333   Z
7           8405 1997-10-27    46    24   27.0   32.333333   Z
8           8405 1997-10-27    64    63   61.0   62.666667   Z
9           8405 1997-10-27    77    82   98.0   85.666667   Z

In [8]:
#bishayee_colony = bishayee_colony["Bate # B0/B00","Date","Inv","col1","col2","col3","average"]
bishayee_colony2 = bishayee_colony.iloc[:,[0,1,6,2,3,4,5]]
bishayee_colony2.columns
bishayee_colony2.head(10)

Bate # B0/B00       Date Inv  col1  col2   col3     average
0           8405 1997-10-27   Z    78    91   93.0   87.333333
1           8405 1997-10-27   Z    90    88   90.0   89.333333
2           8405 1997-10-27   Z    80    66   69.0   71.666667
3           8405 1997-10-27   Z    63    67   71.0   67.000000
4           8405 1997-10-27   Z    44    58   64.0   55.333333
5           8405 1997-10-27   Z    38    53   51.0   47.333333
6           8405 1997-10-27   Z   247   264  258.0  256.333333
7           8405 1997-10-27   Z    46    24   27.0   32.333333
8           8405 1997-10-27   Z    64    63   61.0   62.666667
9           8405 1997-10-27   Z    77    82   98.0   85.666667

In [9]:
bishayee_colony2.rename(columns={'Bate # B0/B00':'Batch'},inplace = True)
bishayee_colony2.drop('Batch', axis=1, inplace=True)
bishayee_colony2.head(10)

Date Inv  col1  col2   col3     average
0 1997-10-27   Z    78    91   93.0   87.333333
1 1997-10-27   Z    90    88   90.0   89.333333
2 1997-10-27   Z    80    66   69.0   71.666667
3 1997-10-27   Z    63    67   71.0   67.000000
4 1997-10-27   Z    44    58   64.0   55.333333
5 1997-10-27   Z    38    53   51.0   47.333333
6 1997-10-27   Z   247   264  258.0  256.333333
7 1997-10-27   Z    46    24   27.0   32.333333
8 1997-10-27   Z    64    63   61.0   62.666667
9 1997-10-27   Z    77    82   98.0   85.666667

In [10]:
others_colony2 = others_colony.rename(columns={'Bates # B00 or B0':'Batch'})
others_colony2.drop('Batch', axis=1, inplace=True)
others_colony2.head(10)

Date Inv  col1   col2 col3     average
0 1992-04-23   I   266  247.0  262  258.333333
1 1992-04-23   I   170  151.0  156  159.000000
2 1992-04-23   I    66   66.0   56   62.666667
3 1992-04-23   I    22   13.0   27   20.666667
4 1992-04-23   I     1    1.0    4    2.000000
5 1992-04-23   I    10   13.0   12   11.666667
6 1992-04-23   I   320  311.0  312  314.333333
7 1992-04-23   I   194  192.0  203  196.333333
8 1992-04-23   I   238  228.0  215  227.000000
9 1992-04-23   I    94   81.0   79   84.666667

In [11]:
outside3_colony["Inv"] = "O"
outside3_colony2=outside3_colony.iloc[:,[0,5,1,2,3,4]]
outside3_colony2.rename(columns={'date':'Date','c1':'col1','c2':'col2','c3':'col3'},inplace = True)
outside3_colony2.head(10)

Date Inv  col1  col2  col3    average
0 2010-02-04   O    54    55    59  56.000000
1        NaT   O    47    60    47  51.333333
2 2010-02-05   O    55    60    53  56.000000
3        NaT   O    58    54    59  57.000000
4        NaT   O    17    17    15  16.333333
5 2012-02-12   O    65    64    55  61.333333
6 2012-02-15   O    64    57    73  64.666667
7        NaT   O    84   109    89  94.000000
8 2012-02-17   O    64    64    62  63.333333
9        NaT   O    68    57    68  64.333333

# Combining the Colony Data

In [12]:
df_total = [bishayee_colony2,others_colony2,outside3_colony2]
merged_colony_data = pd.concat(df_total)
merged_colony_data.size == bishayee_colony2.size + others_colony2.size + outside3_colony2.size

True

In [13]:
merged_colony_data.head(5)

Date Inv  col1  col2 col3    average
0 1997-10-27   Z    78  91.0   93  87.333333
1 1997-10-27   Z    90  88.0   90  89.333333
2 1997-10-27   Z    80  66.0   69  71.666667
3 1997-10-27   Z    63  67.0   71  67.000000
4 1997-10-27   Z    44  58.0   64  55.333333

In [14]:
bishayee_coulter.head(5)

Bates       Date  Count 1  Count 2  Count 3     Average
0  B008404 1997-10-20      531      508    541.0  526.666667
1  B008404 1997-10-20      650      626    595.0  623.666667
2  B008404 1997-10-20      460      455    468.0  461.000000
3  B008404 1997-10-20      550      530    538.0  539.333333
4  B008404 1997-10-20      466      468    452.0  462.000000

In [15]:
bishayee_coulter2 = bishayee_coulter.drop('Bates', axis=1)
bishayee_coulter2["Inv"] = "Z"
bishayee_coulter2 = bishayee_coulter2.iloc[:,[0,5,1,2,3,4]]
bishayee_coulter2.rename(columns={'Count 1':'col1','Count 2':'col2','Count 3':'col3'},inplace=True)
bishayee_coulter2.head(5)

Date Inv  col1  col2   col3     Average
0 1997-10-20   Z   531   508  541.0  526.666667
1 1997-10-20   Z   650   626  595.0  623.666667
2 1997-10-20   Z   460   455  468.0  461.000000
3 1997-10-20   Z   550   530  538.0  539.333333
4 1997-10-20   Z   466   468  452.0  462.000000

In [16]:
outside1_coulter.head(5)

Experiment    C1    C2      C3      Average       Date
0      Exp 1  2266  2321  2192.0  2259.666667 1991-06-07
1        NaN   676   554   478.0   569.333333        NaT
2        NaN   601   723   520.0   614.666667        NaT
3        NaN   582   516   881.0   659.666667        NaT
4        NaN  3200  2719  3747.0  3222.000000        NaT

In [17]:
outside1_coulter2 = outside1_coulter.drop('Experiment', axis = 1)
outside1_coulter2["Inv"] = "O1"
outside1_coulter2 = outside1_coulter2.iloc[:,[4,5,0,1,2,3]]
outside1_coulter2.rename(columns={'C1':'col1','C2':'col2','C3':'col3'},inplace=True)
outside1_coulter2.head(5)

Date Inv  col1  col2    col3      Average
0 1991-06-07  O1  2266  2321  2192.0  2259.666667
1        NaT  O1   676   554   478.0   569.333333
2        NaT  O1   601   723   520.0   614.666667
3        NaT  O1   582   516   881.0   659.666667
4        NaT  O1  3200  2719  3747.0  3222.000000

In [18]:
outside2_coulter.head(5)

Date  Count 1  Count 2  Count 3      Average
0  6.6.2008     5868     5838     5691  5799.000000
1       NaN     3451     3343     3315  3369.666667
2       NaN     4844     4854     4695  4797.666667
3   6.10.08     4851     4549     4532  4644.000000
4       NaN     3010     3018     2982  3003.333333

In [19]:
outside2_coulter2 = outside2_coulter.rename(columns={'Count 1':"col1",'Count 2':"col2",'Count 3':"col3"})
outside2_coulter2["Inv"] = "O2"
outside2_coulter2 = outside2_coulter2.iloc[:,[0,5,1,2,3,4]]
outside2_coulter2.head(5)

Date Inv  col1  col2  col3      Average
0  6.6.2008  O2  5868  5838  5691  5799.000000
1       NaN  O2  3451  3343  3315  3369.666667
2       NaN  O2  4844  4854  4695  4797.666667
3   6.10.08  O2  4851  4549  4532  4644.000000
4       NaN  O2  3010  3018  2982  3003.333333

In [20]:
others_coulter.head(5)

Bates No.       Date  Coul 1  Coul 2  Coul 3      Average Investigator
0       NaN 1992-04-15  1547.0  1574.0  1523.0  1548.000000            I
1       NaN 1992-04-15  1617.0  1552.0  1570.0  1579.666667            I
2       NaN 1992-04-15  1258.0  1279.0  1284.0  1273.666667            I
3       NaN 1992-04-15  1273.0  1313.0  1286.0  1290.666667            I
4       NaN 1992-04-15  1071.0  1044.0  1044.0  1053.000000            I

In [21]:
others_coulter2 = others_coulter.drop('Bates No.', axis = 1)
others_coulter2.rename(columns={'Coul 1':'col1','Coul 2':'col2','Coul 3':'col3','Investigator':'Inv'},\
                       inplace=True)
others_coulter2 = others_coulter2.iloc[:,[0,5,1,2,3,4]]
others_coulter2.head(5)


Date Inv    col1    col2    col3      Average
0 1992-04-15   I  1547.0  1574.0  1523.0  1548.000000
1 1992-04-15   I  1617.0  1552.0  1570.0  1579.666667
2 1992-04-15   I  1258.0  1279.0  1284.0  1273.666667
3 1992-04-15   I  1273.0  1313.0  1286.0  1290.666667
4 1992-04-15   I  1071.0  1044.0  1044.0  1053.000000

# Combine the Coulter Data

In [22]:
df_total_coulter = [bishayee_coulter2,others_coulter2,outside1_coulter2,outside2_coulter2]
merged_coulter_data = pd.concat(df_total_coulter)
merged_coulter_data.size == bishayee_coulter2.size + others_coulter2.size + outside1_coulter2.size \
+ outside2_coulter2.size

True

In [23]:
merged_coulter_data.head(10)

Date Inv   col1   col2   col3     Average
0  1997-10-20 00:00:00   Z  531.0  508.0  541.0  526.666667
1  1997-10-20 00:00:00   Z  650.0  626.0  595.0  623.666667
2  1997-10-20 00:00:00   Z  460.0  455.0  468.0  461.000000
3  1997-10-20 00:00:00   Z  550.0  530.0  538.0  539.333333
4  1997-10-20 00:00:00   Z  466.0  468.0  452.0  462.000000
5  1997-10-20 00:00:00   Z  567.0  555.0  521.0  547.666667
6  1997-10-20 00:00:00   Z  558.0  581.0  636.0  591.666667
7  1997-10-20 00:00:00   Z  567.0  563.0  537.0  555.666667
8  1997-10-20 00:00:00   Z  594.0  550.0  543.0  562.333333
9  1997-10-20 00:00:00   Z  611.0  599.0  507.0  572.333333

In [24]:
merged_coulter_data["LowerRatio"] = 0
merged_coulter_data["UpperRatio"] = 0
merged_coulter_data.drop("Date",axis =1,inplace=True)
merged_coulter_data=merged_coulter_data.dropna()
merged_coulter_data.reset_index(drop=True)
merged_coulter_data.head(10)

Inv   col1   col2   col3     Average  LowerRatio  UpperRatio
0   Z  531.0  508.0  541.0  526.666667           0           0
1   Z  650.0  626.0  595.0  623.666667           0           0
2   Z  460.0  455.0  468.0  461.000000           0           0
3   Z  550.0  530.0  538.0  539.333333           0           0
4   Z  466.0  468.0  452.0  462.000000           0           0
5   Z  567.0  555.0  521.0  547.666667           0           0
6   Z  558.0  581.0  636.0  591.666667           0           0
7   Z  567.0  563.0  537.0  555.666667           0           0
8   Z  594.0  550.0  543.0  562.333333           0           0
9   Z  611.0  599.0  507.0  572.333333           0           0

In [25]:
for i in range(0,len(merged_coulter_data)):
    temp_array = np.ones(3)
    for k in range(0,3):
        temp_array[k] = merged_coulter_data.iloc[i,(1+k)]
    temp_array = np.sort(temp_array)
    temp_l = (temp_array[1]-temp_array[0])/(temp_array[2]-temp_array[0])
    temp_u = (temp_array[2]-temp_array[1])/(temp_array[2]-temp_array[0])
    merged_coulter_data.iloc[i,5] = temp_l
    merged_coulter_data.iloc[i,6] = temp_u
    if (merged_coulter_data.iloc[i,5]) == 0 and (merged_coulter_data.iloc[i,6]) != 1:
        print("data issues")

In [26]:
merged_coulter_data.head(5)

Inv   col1   col2   col3     Average  LowerRatio  UpperRatio
0   Z  531.0  508.0  541.0  526.666667    0.696970    0.303030
1   Z  650.0  626.0  595.0  623.666667    0.563636    0.436364
2   Z  460.0  455.0  468.0  461.000000    0.384615    0.615385
3   Z  550.0  530.0  538.0  539.333333    0.400000    0.600000
4   Z  466.0  468.0  452.0  462.000000    0.875000    0.125000

In [27]:
merged_colony_data["LowerRatio"] = 0
merged_colony_data["UpperRatio"] = 0
merged_colony_data.drop("Date",axis =1,inplace=True)
merged_colony_data.replace(r'\s*',np.nan, regex=True )
merged_colony_data=merged_colony_data.dropna()
merged_colony_data.reset_index(drop=True)
messup_rows=[]
for i in range(0,len(merged_colony_data)):
    temp_array = np.ones(3)
    for k in range(0,3):
        #print(merged_colony_data.iloc[i,(1+k)],i)
        if type(merged_colony_data.iloc[i,(1+k)]) == str:
            messup_rows.append(i)
            continue
        else:
            temp_array[k] = merged_colony_data.iloc[i,(1+k)]
    temp_array = np.sort(temp_array)
    temp_l = (temp_array[1]-temp_array[0])/(temp_array[2]-temp_array[0])
    temp_u = (temp_array[2]-temp_array[1])/(temp_array[2]-temp_array[0])
    merged_colony_data.iloc[i,5] = temp_l
    merged_colony_data.iloc[i,6] = temp_u
merged_colony_data=merged_colony_data.drop(messup_rows)
for i in range(0,len(merged_colony_data)):
    if (merged_colony_data.iloc[i,5]) == 0 and (merged_colony_data.iloc[i,6]) != 1:
        print("data issues")

In [28]:
merged_colony_data.head(5)

Inv  col1  col2 col3    average  LowerRatio  UpperRatio
0   Z    78  91.0   93  87.333333    0.866667    0.133333
1   Z    90  88.0   90  89.333333    1.000000    0.000000
2   Z    80  66.0   69  71.666667    0.214286    0.785714
3   Z    63  67.0   71  67.000000    0.500000    0.500000
4   Z    44  58.0   64  55.333333    0.700000    0.300000